# Продвинутые примеры: Нетривиальные системы

В этом туториале мы исследуем **сложные системы переписывания** с интересными математическими свойствами.

## 🎯 Что вы изучите

1. **Арифметическая система** — правила для унарной арифметики
2. **Циклическая система** — правила с нетривиальными ω-пределами
3. **Хаотическая система** — правила с высокой энтропией
4. **Сравнительный анализ** — метрики разных систем

## 🔬 Цель

Понять, как структура правил влияет на:
- Размер достижимого множества
- Топологию графа переходов
- Геометрические характеристики
- Вероятностные распределения

In [27]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

from rewriting import String, Rule, RewritingEngine
from integration import SpaceLanguageOrchestrator, PipelineConfig

print("✓ Модули загружены")

✓ Модули загружены


## 🚀 Настройка окружения

In [28]:
# Арифметические правила
arithmetic_rules = [
    # Сложение: 0+0 = 00 (удаляем +)
    Rule(String.from_str("0|0"), String.from_str("00")),
    
    # Умножение: 0*| = 0 (умножение на 1)
    Rule(String.from_str("0||"), String.from_str("0")),
    
    # Упрощение: 00 → |0 (группировка двоек)
    Rule(String.from_str("00"), String.from_str("|0")),
    
    # Нормализация: || → 0 (две единицы = два)
    Rule(String.from_str("||"), String.from_str("0")),
]

print("Арифметическая система:")
for i, rule in enumerate(arithmetic_rules, 1):
    print(f"  R{i}: {rule}")

# Тестовая строка: "0000" = 4
initial_arith = String.from_str("0000")
print(f"\n✓ Начальная строка: {initial_arith} (число 4)")

Арифметическая система:
  R1: 0|0 → 00
  R2: 0|| → 0
  R3: 00 → |0
  R4: || → 0

✓ Начальная строка: 0000 (число 4)


## Пример 1️⃣: Арифметическая система

Система правил для унарной арифметики — сложение и умножение чисел в унарном представлении:

In [29]:
# Анализ арифметической системы
config_arith = PipelineConfig(
    graph_depth=8,  # Увеличиваем глубину
    max_graph_size=200,
    enable_geometry=True,
    enable_measure=True,
    embedding_dim=8
)

orch_arith = SpaceLanguageOrchestrator(arithmetic_rules, config_arith)
result_arith = orch_arith.analyze(initial_arith, verbose=False)

print(f"✓ Арифметическая система проанализирована")
print(f"  Достижимо: {len(result_arith.reachable_strings)} строк")
print(f"  Граф: {result_arith.graph_vertices} вершин, {result_arith.graph_edges} рёбер")
if result_arith.measure_data:
    print(f"  Энтропия: {result_arith.measure_data.get('entropy', 0):.3f} бит")

✓ Арифметическая система проанализирована
  Достижимо: 14 строк
  Граф: 14 вершин, 22 рёбер
  Энтропия: 3.807 бит


## Пример 2️⃣: Циклическая система

Система с выраженными циклами и периодическим поведением. Правила образуют замкнутые петли переходов — идеально для изучения SCC (сильно связанных компонент):

In [37]:
# Циклические правила
cyclic_rules = [
    Rule(String.from_str("0"), String.from_str("|")),    # 0 → |
    Rule(String.from_str("|"), String.from_str("0|")),   # | → 0|
    Rule(String.from_str("0|"), String.from_str("||")),  # 0| → ||
    Rule(String.from_str("||"), String.from_str("0")),   # || → 0
]

print("Циклическая система:")
for i, rule in enumerate(cyclic_rules, 1):
    print(f"  R{i}: {rule}")

# Начальное состояние
initial_cyclic = String.from_str("0")
print(f"\n✓ Начальная строка: {initial_cyclic}")

Циклическая система:
  R1: 0 → |
  R2: | → 0|
  R3: 0| → ||
  R4: || → 0

✓ Начальная строка: 0


### Анализ циклической системы

Исследуем ω-предел (предельное множество) циклической системы и её структуру:

In [38]:
# Исследуем цикл
engine_cyclic = RewritingEngine(cyclic_rules)
omega_cyclic = engine_cyclic.omega_limit(initial_cyclic, max_steps=20)

print(f"ω-предел циклической системы ({len(omega_cyclic)} строк):")
for s in sorted(omega_cyclic, key=str)[:10]:  # Показываем только первые 10
    print(f"  {s}")
if len(omega_cyclic) > 10:
    print(f"  ... ещё {len(omega_cyclic) - 10} строк")

# Полный анализ с меньшей глубиной из-за роста
config_cyclic = PipelineConfig(
    graph_depth=3,  # Меньше из-за роста
    max_graph_size=50,
    enable_measure=True
)
orch_cyclic = SpaceLanguageOrchestrator(cyclic_rules, config_cyclic)
result_cyclic = orch_cyclic.analyze(initial_cyclic, verbose=False)

print(f"\n✓ Циклическая система проанализирована")
print(f"  Достижимо: {len(result_cyclic.reachable_strings)} строк")
print(f"  Граф: {result_cyclic.graph_vertices} вершин")
print(f"  SCC: {result_cyclic.scc_count} компонент")
print(f"  Наибольшая SCC: {result_cyclic.largest_scc_size} вершин")

ω-предел циклической системы (20 строк):
  0
  0|
  0||
  0|||
  0||||
  0|||||
  0||||||
  0|||||||
  0||||||||
  0|||||||||
  ... ещё 10 строк

✓ Циклическая система проанализирована
  Достижимо: 24 строк
  Граф: 24 вершин
  SCC: 8 компонент
  Наибольшая SCC: 17 вершин


## Пример 3️⃣: Хаотическая система

Система с высокой энтропией и экспоненциальным ростом числа состояний. Множество правил с недетерминизмом создаёт сложное, непредсказуемое поведение:

In [39]:
# Хаотические правила (много недетерминизма)
chaotic_rules = [
    Rule(String.from_str("0"), String.from_str("|")),
    Rule(String.from_str("0"), String.from_str("||")),
    Rule(String.from_str("|"), String.from_str("0")),
    Rule(String.from_str("|"), String.from_str("00")),
    Rule(String.from_str("00"), String.from_str("|")),
    Rule(String.from_str("||"), String.from_str("0")),
]

print("Хаотическая система:")
for i, rule in enumerate(chaotic_rules, 1):
    print(f"  R{i}: {rule}")

initial_chaotic = String.from_str("0|")
print(f"\n✓ Начальная строка: {initial_chaotic}")

Хаотическая система:
  R1: 0 → |
  R2: 0 → ||
  R3: | → 0
  R4: | → 00
  R5: 00 → |
  R6: || → 0

✓ Начальная строка: 0|


In [40]:
# Анализ хаотической системы
config_chaotic = PipelineConfig(
    graph_depth=5,
    max_graph_size=150,
    enable_measure=True,
    embedding_dim=12
)

orch_chaotic = SpaceLanguageOrchestrator(chaotic_rules, config_chaotic)
result_chaotic = orch_chaotic.analyze(initial_chaotic, verbose=False)

print(f"✓ Хаотическая система проанализирована")
print(f"  Достижимо: {len(result_chaotic.reachable_strings)} строк")
print(f"  Граф: {result_chaotic.graph_vertices} вершин, {result_chaotic.graph_edges} рёбер")
if result_chaotic.measure_data:
    entropy = result_chaotic.measure_data.get('entropy', 0)
    print(f"  Энтропия: {entropy:.3f} бит (высокая хаотичность!)")

✓ Хаотическая система проанализирована
  Достижимо: 5458 строк
  Граф: 5458 вершин, 92834 рёбер
  Энтропия: 12.414 бит (высокая хаотичность!)


## 📊 Сравнительный анализ

Сведём метрики всех трёх систем в единую таблицу и сравним их характеристики по различным параметрам:

In [41]:
# Собираем данные для сравнения
systems_data = {
    "Система": ["Арифметическая", "Циклическая", "Хаотическая"],
    "Правил": [len(arithmetic_rules), len(cyclic_rules), len(chaotic_rules)],
    "Достижимо": [
        len(result_arith.reachable_strings),
        len(result_cyclic.reachable_strings),
        len(result_chaotic.reachable_strings)
    ],
    "Вершин": [
        result_arith.graph_vertices,
        result_cyclic.graph_vertices,
        result_chaotic.graph_vertices
    ],
    "Рёбер": [
        result_arith.graph_edges,
        result_cyclic.graph_edges,
        result_chaotic.graph_edges
    ],
    "SCC": [
        result_arith.scc_count,
        result_cyclic.scc_count,
        result_chaotic.scc_count
    ],
    "Энтропия": [
        result_arith.measure_data.get('entropy', 0) if result_arith.measure_data else 0,
        result_cyclic.measure_data.get('entropy', 0) if result_cyclic.measure_data else 0,
        result_chaotic.measure_data.get('entropy', 0) if result_chaotic.measure_data else 0
    ]
}

# Форматированный вывод таблицы
print("СРАВНИТЕЛЬНАЯ ТАБЛИЦА")
print("="*80)
print(f"{'Система':<20} {'Правил':>8} {'Достижимо':>12} {'Вершин':>10} {'Рёбер':>10} {'SCC':>6} {'Энтропия':>10}")
print("-"*80)
for i in range(len(systems_data["Система"])):
    print(f"{systems_data['Система'][i]:<20} "
          f"{systems_data['Правил'][i]:>8} "
          f"{systems_data['Достижимо'][i]:>12} "
          f"{systems_data['Вершин'][i]:>10} "
          f"{systems_data['Рёбер'][i]:>10} "
          f"{systems_data['SCC'][i]:>6} "
          f"{systems_data['Энтропия'][i]:>10.3f}")
print("="*80)

СРАВНИТЕЛЬНАЯ ТАБЛИЦА
Система                Правил    Достижимо     Вершин      Рёбер    SCC   Энтропия
--------------------------------------------------------------------------------
Арифметическая              4           14         14         22     14      3.807
Циклическая                 4           24         24         60      8      4.585
Хаотическая                 6         5458       5458      92834      0     12.414


## 🔬 Геометрические характеристики

Сравним геометрические лифты систем — размерности многообразий, кривизну и количество точек в геометрическом представлении:

In [42]:
print("ГЕОМЕТРИЧЕСКИЕ ХАРАКТЕРИСТИКИ")
print("="*80)

for name, result in [
    ("Арифметическая", result_arith),
    ("Циклическая", result_cyclic),
    ("Хаотическая", result_chaotic)
]:
    print(f"\n{name} система:")
    
    if result.geometry_data:
        dim = result.geometry_data.get('dimension', 'N/A')
        curv = result.geometry_data.get('curvature', 0)
        pts = result.geometry_data.get('points', 0)
        print(f"  📐 Размерность: {dim}")
        print(f"  📐 Кривизна: {curv:.4f}")
        print(f"  📐 Точек: {pts}")
    else:
        print("  (геометрия не вычислена)")
        
print("="*80)

ГЕОМЕТРИЧЕСКИЕ ХАРАКТЕРИСТИКИ

Арифметическая система:
  📐 Размерность: 1
  📐 Кривизна: 0.0472
  📐 Точек: 14

Циклическая система:
  📐 Размерность: 2
  📐 Кривизна: 0.0589
  📐 Точек: 20

Хаотическая система:
  📐 Размерность: 2
  📐 Кривизна: 0.0482
  📐 Точек: 20


## 💡 Ключевые наблюдения

На основе сравнительного анализа выделим закономерности и важные выводы о поведении систем переписывания:

In [43]:
print("КЛЮЧЕВЫЕ НАБЛЮДЕНИЯ")
print("="*80)

# Энтропии
entropies = [
    ("Арифметическая", result_arith.measure_data.get('entropy', 0) if result_arith.measure_data else 0),
    ("Циклическая", result_cyclic.measure_data.get('entropy', 0) if result_cyclic.measure_data else 0),
    ("Хаотическая", result_chaotic.measure_data.get('entropy', 0) if result_chaotic.measure_data else 0)
]

max_entropy_sys = max(entropies, key=lambda x: x[1])
min_entropy_sys = min(entropies, key=lambda x: x[1])

print(f"\n1. Самая упорядоченная: {min_entropy_sys[0]} (энтропия {min_entropy_sys[1]:.3f} бит)")
print(f"   → Низкая энтропия = высокая предсказуемость")

print(f"\n2. Самая хаотичная: {max_entropy_sys[0]} (энтропия {max_entropy_sys[1]:.3f} бит)")
print(f"   → Высокая энтропия = много вариантов развития")

# SCC анализ
print(f"\n3. Циклическая структура:")
print(f"   - Циклическая система: {result_cyclic.largest_scc_size} вершин в наибольшей SCC")
print(f"   - Указывает на замкнутые аттракторы")

# Размер графа
print(f"\n4. Сложность графа:")
print(f"   - Хаотическая: {result_chaotic.graph_edges} рёбер (самая связная)")
print(f"   - Арифметическая: {result_arith.graph_edges} рёбер")
print(f"   - Больше рёбер = больше недетерминизма")

print("="*80)

КЛЮЧЕВЫЕ НАБЛЮДЕНИЯ

1. Самая упорядоченная: Арифметическая (энтропия 3.807 бит)
   → Низкая энтропия = высокая предсказуемость

2. Самая хаотичная: Хаотическая (энтропия 12.414 бит)
   → Высокая энтропия = много вариантов развития

3. Циклическая структура:
   - Циклическая система: 17 вершин в наибольшей SCC
   - Указывает на замкнутые аттракторы

4. Сложность графа:
   - Хаотическая: 92834 рёбер (самая связная)
   - Арифметическая: 22 рёбер
   - Больше рёбер = больше недетерминизма


## 📊 Резюме

В этом туториале вы исследовали:

✅ **Арифметическую систему** — правила для унарной арифметики  
✅ **Циклическую систему** — периодическое поведение с выраженными аттракторами  
✅ **Хаотическую систему** — высокая энтропия и недетерминизм  
✅ **Сравнительный анализ** — количественное сравнение по метрикам  

## 🎯 Ключевые выводы

1. **Структура правил определяет поведение**: небольшие изменения в правилах драматически влияют на динамику
2. **Энтропия как мера сложности**: высокая энтропия = много вариантов, низкая = детерминизм
3. **SCC указывают на аттракторы**: большие компоненты = замкнутые циклы
4. **Геометрия отражает структуру**: кривизна и размерность зависят от правил

## 🚀 Эксперименты

Попробуйте создать собственные системы:

```python
my_rules = [
    Rule(String.from_str("..."), String.from_str("...")),
    # Добавьте свои правила
]
my_orchestrator = SpaceLanguageOrchestrator(my_rules, config)
my_result = my_orchestrator.analyze(initial_string)
```

## 📚 Что дальше?

- Изучите примеры в `examples/`
- Прочитайте полную документацию в [INTEGRATION.md](../INTEGRATION.md)
- Экспериментируйте с параметрами `PipelineConfig`